# 1.5 JOINs y Relaciones entre Tablas

- Objetivo: combinar datos de múltiples tablas usando claves primarias y foráneas.
- Prerrequisitos: entender modelo relacional (fact_ventas referencia dim_clientes, dim_productos).
- Ejercicios: INNER, LEFT JOIN para detectar faltantes, múltiples JOINs.
- Reto: agregación sobre JOINs con LEFT para incluir regiones sin actividad.
- Errores comunes: INNER cuando se necesitan faltantes, duplicar filas por JOIN incorrecto.

## ¿Por qué usar JOINs?

Los datos normalizados se almacenan en múltiples tablas para evitar redundancia. Por ejemplo:
- `fact_ventas` guarda solo `cliente_id` (número), no el nombre del cliente
- Para mostrar "Juan Pérez compró Laptop X", necesitamos **unir** fact_ventas con dim_clientes y dim_productos

**Para qué:**
- Desnormalizar datos para reportes legibles
- Enriquecer fact tables con atributos de dimensiones (categoría, región, segmento)
- Detectar registros huérfanos (ventas sin cliente válido)

**Cómo:**
- **INNER JOIN:** Solo registros que coinciden en ambas tablas (ventas con cliente existente)
- **LEFT JOIN:** Todos los registros de tabla izquierda + coincidencias de derecha (productos con o sin ventas)
- **RIGHT JOIN:** Inversa de LEFT (raro, mejor reescribir como LEFT)
- **FULL OUTER JOIN:** Todos los registros de ambas tablas (útil para consolidaciones)

## Tipos de JOIN

### INNER JOIN (Intersección)
```sql
SELECT v.venta_id, c.nombre
FROM fact_ventas v
INNER JOIN dim_clientes c ON v.cliente_id = c.cliente_id;
```
**Resultado:** Solo ventas que tienen cliente válido. Si hay cliente_id=999 en ventas pero no existe en dim_clientes, esa venta NO aparece.

### LEFT JOIN (Izquierda + Coincidencias)
```sql
SELECT p.nombre, v.venta_id
FROM dim_productos p
LEFT JOIN fact_ventas v ON p.producto_id = v.producto_id;
```
**Resultado:** Todos los productos (incluso sin ventas). Si un producto no tiene ventas, venta_id será NULL.

**Detectar productos sin ventas:**
```sql
WHERE v.venta_id IS NULL  -- Filtra solo productos sin coincidencias
```

### Múltiples JOINs
```sql
SELECT v.venta_id, c.nombre AS cliente, p.nombre AS producto, r.nombre AS region
FROM fact_ventas v
JOIN dim_clientes c ON v.cliente_id = c.cliente_id
JOIN dim_productos p ON v.producto_id = p.producto_id
JOIN dim_regiones r ON c.region_id = r.region_id;
```
**Lectura:** "Cada venta conecta con su cliente, producto y región del cliente"

## Errores Comunes

1. **Usar INNER cuando se necesitan faltantes:** Si el objetivo es "listar todos los productos, incluyendo los sin ventas", INNER JOIN eliminará productos sin ventas. Usar LEFT JOIN.

2. **Duplicar filas por JOIN mal definido:** Si dim_clientes tiene registros duplicados o la condición ON es ambigua, cada venta puede multiplicarse.

3. **No calificar columnas en JOINs múltiples:** Si `nombre` existe en dim_clientes y dim_productos, escribir `c.nombre` y `p.nombre` para evitar ambigüedad.

4. **Olvidar IS NULL para detectar faltantes:** Después de LEFT JOIN, filtrar `WHERE tabla_derecha.id IS NULL` para encontrar registros sin coincidencia.

In [ ]:
-- Ventas con cliente y producto (INNER JOIN)
SELECT v.venta_id, v.fecha, c.nombre AS cliente, p.nombre AS producto, v.cantidad,
       (v.cantidad * p.precio_unitario) AS ingreso_bruto
FROM dbo.fact_ventas v
JOIN dbo.dim_clientes c ON v.cliente_id = c.cliente_id
JOIN dbo.dim_productos p ON v.producto_id = p.producto_id;

🟢 Ejercicio: Ventas con región del cliente (JOIN adicional).

In [ ]:
SELECT v.venta_id, r.nombre AS region, v.cantidad
FROM dbo.fact_ventas v
JOIN dbo.dim_clientes c ON v.cliente_id = c.cliente_id
JOIN dbo.dim_regiones r ON c.region_id = r.region_id;

🟠 Ejercicio: Productos sin ventas (usar LEFT JOIN y filtro NULL).

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_ventas v ON p.producto_id = v.producto_id
WHERE v.producto_id IS NULL;

🔴 Reto: Contar ventas por región mostrando también regiones sin ventas (LEFT).

In [ ]:
SELECT r.nombre AS region, COUNT(v.venta_id) AS ventas
FROM dbo.dim_regiones r
LEFT JOIN dbo.dim_clientes c ON r.region_id = c.region_id
LEFT JOIN dbo.fact_ventas v ON c.cliente_id = v.cliente_id
GROUP BY r.nombre
ORDER BY ventas DESC;

Errores comunes: usar INNER cuando se requieren faltantes; duplicar filas por JOIN mal definido; no calificar columnas en joins múltiples.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.